# 뉴스 헤드라인 분류하기 (로컬 버전)

> 이 노트북은 세이지메이커 스튜디오 상에서`Python 3 (PyTorch 1.13 Python 3.9 CPU Optimized)` 커널을 사용하지면 잘 작동합니다.

이 예에서는 사용자 지정 스크립트와 [Hugging Face Transformers](https://huggingface.co/docs/transformers/index) 프레임워크를 사용하여 뉴스 헤드라인 분류 모델을 훈련합니다.

이 "로컬" 노트북은 여기 노트북 자체에서 모델을 훈련하고 테스트하는 데모를 보여줄 것이며, 동반되는 ["SageMaker" 노트북](Headline%20Classifier%20SageMaker.ipynb)은 컨테이너화된 SageMaker 훈련 작업과 엔드포인트 배포를 사용하여 동일한 프로세스를 반복할 것입니다.

허깅 페이스를 처음 사용하는 경우 [Transformers quick tour](https://huggingface.co/docs/transformers/quicktour)를 읽어보거나 다음 소개 동영상(1시간)을 시청하는 것이 좋습니다:

In [ ]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/pYqjCzoyWyo" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>

## 설치 및 설정

위에 명시된 파이토치 세이지메이커 커널에는 필요한 라이브러리가 대부분 포함되어 있지만 모든 라이브러리가 포함되어 있지는 않습니다. 먼저 적절한 버전의 HF transformers/datasets을 설치해야 하며, 나중에 대화형 분류 위젯을 구동하기 위해 IPyWidgets도 설치해야 합니다:

> ⚠️ **Note:** 이 셀을 먼저 실행하는 한 노트북 커널을 재시작할 필요가 없습니다. 하지만 이미 어떤 것이라도 'import'를 한 경우, 위의 도구 모음에서 'restart the kernel' 버튼을 클릭해야 설치가 적용됩니다.

아래 출력에서 pip의 *warnings*는 무시할 수 있지만 *errors*는 표시되지 않아야 합니다.

In [ ]:
%pip install datasets "ipywidgets<8" transformers==4.26

설치가 완료되면 나머지 노트북에서 사용할 라이브러리와 Python 내장 기능을 로드합니다.

[%autoreload magic](https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html)은 로컬 .py 파일로 작업할 때 유용합니다. 셀을 실행할 때마다 라이브러리를 다시 로드하면 노트북 커널을 재시작할 필요 없이 로컬에서 편집/업데이트된 스크립트를 사용할 수 있기 때문입니다.

In [ ]:
%load_ext autoreload
%autoreload 2

# Python Built-Ins:
import os  # Operating system utils e.g. file paths

# External Dependencies:
import datasets  # Hugging Face data loading utilities
import ipywidgets as widgets  # Interactive prediction widget
import pandas as pd  # Utilities for working with data tables (dataframes)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import transformers  # Hugging Face Transformers framework

local_dir = "data"

## 데이터셋 준비

이 예제에서는 [Registry of Open Data on AWS](https://registry.opendata.aws/fast-ai-nlp/) 퍼블릭 리포지토리에서 **FastAi AG News** 데이터 세트를 다운로드합니다. 이 데이터 세트에는 뉴스 헤드라인과 그에 해당하는 주제 클래스의 표가 포함되어 있습니다.

In [ ]:
%%time
# Download the AG News data from the Registry of Open Data on AWS.
!mkdir -p {local_dir}
!aws s3 cp s3://fast-ai-nlp/ag_news_csv.tgz {local_dir} --no-sign-request

# Un-tar the AG News data.
!tar zxf {local_dir}/ag_news_csv.tgz -C {local_dir}/ --strip-components=1 --no-same-owner
print("Done!")

데이터를 다운로드하고 추출한 후 아래와 같이 몇 가지 예를 살펴볼 수 있습니다:

In [ ]:
column_names = ["CATEGORY", "TITLE", "CONTENT"]
# we use the train.csv only
df = pd.read_csv(f"{local_dir}/train.csv", names=column_names, header=None, delimiter=",")
# shuffle the DataFrame rows
df = df.sample(frac=1, random_state=1337)

# Make the (1-indexed) category classes more readable:
class_names = ["Other", "World", "Sports", "Business", "Sci/Tech"]
idx2label = {ix: name for ix, name in enumerate(class_names)}
label2idx = {name: ix for ix, name in enumerate(class_names)}

df = df.replace({"CATEGORY": idx2label})
df.head()

이번 연습에서는 **아래의 값만 사용하겠습니다**:

- 뉴스 기사의 **title**(Headline)을 입력으로 사용합니다.
- 예측할 목표 변수로 **category**를 사용합니다.

이 데이터 세트에는 아래와 같이 4개의 균등하게 분포된 토픽 클래스가 포함되어 있습니다.

> ℹ️ **'Other'는 어떻게 할까요?:** 원시 데이터 세트는 1~4 사이의 숫자로 범주를 나타내며, 우리의 모델은 0부터 시작하는 숫자를 예상하기 때문에, 데이터 준비를 단순하게 유지하고 클래스의 혼란스러운 추가 숫자 표현을 피하기 위해 사용하지 않는 'Other' 클래스를 삽입했습니다.

In [ ]:
df["CATEGORY"].value_counts()

## 훈련 파라미터 정의

[Hugging Face Hub](https://huggingface.co/models)에서 (비교적 작은) 사전 학습된 모델을 미세 조정하고, 낮은 수준의 학습 루프를 처음부터 작성하는 대신 높은 수준의 [Trainer API](https://huggingface.co/docs/transformers/main_classes/trainer)를 사용할 것입니다.

아래에서는 학습을 위한 기본 파라미터를 설정하겠습니다.

> 🏎️ 이 노트북 내 예제에서는 기본적으로 **저렴한 CPU 전용 컴퓨팅**을 사용하겠습니다. 우리가 훈련하는 모델은 최신 LLM 표준에 따르면 "소규모"이지만, 합리적인 시간 내에 완료할 수 있도록 훈련을 매우 일찍 중단해야 합니다.
>
> 결과 모델은 학습이 매우 부족할 것이며, 동일한 아키텍처가 궁극적으로 도달할 수 있는 것보다 훨씬 덜 정확할 것입니다.

In [ ]:
model_id = "amazon/bort"  # ID of the pre-trained model to start from

training_args = transformers.TrainingArguments(
    output_dir=f"{local_dir}/model",  # Where to save trained model snapshots
    #logging_dir=f"{local_dir}/local-logs",  # Optionally, save logs too
    max_steps=500,  # Maximum number of training steps to run
    num_train_epochs=3,  # Maximum number of times to loop through the training data
    per_device_train_batch_size=16,  # Examples per mini-batch for training
    per_device_eval_batch_size=32,  # Examples per mini-batch for validation
    evaluation_strategy="steps",  # Run validation every N 'steps' instead of every 'epoch'
    eval_steps=100,  # Number of training steps between validation runs
    save_strategy="steps",  # Must be same as evaluation_strategy when load_best_model_at_end=True
    load_best_model_at_end=True,  # If current model at end is not the best, load the best
    metric_for_best_model="f1",  # Use F1 score for judging which model is 'best'
    learning_rate=5e-5,  # Initial learning rate (decays over time by default)
    warmup_steps=100,  # Number of steps to gradually increase the learning rate from the start
)

## 메트릭 정의

여기서는 모델이 검증될 때마다 실행되는 [callback function](https://huggingface.co/docs/transformers/main_classes/callback)를 설정하여 학습된 모델의 품질을 측정하는 방법을 정의하겠습니다.

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="micro")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

## 모델 학습 및 유효성 검사

이 섹션에서는 기본 모델과 데이터 세트를 로드하고 실제 훈련 및 유효성 검사 프로세스를 실행하겠습니다.

먼저, 주어진 'model_id'에 대해 사전 학습된 모델과 함께 제공되는 [tokenizer](https://huggingface.co/docs/transformers/main_classes/tokenizer)를 로드해야 하며, 이는 허깅 페이스 허브에서 자동으로 다운로드됩니다.

모델을 설정하는 과정에서 미세 조정할 토픽 클래스의 수를 지정하고 사람이 읽을 수 있는 이름을 설정해야 합니다:

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

model = transformers.AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=len(class_names))
model.config.label2id = label2idx
model.config.id2label = idx2label

data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

데이터 세트에 이미 제공된 원시 `train.csv` 및 `test.csv` 파일을 훈련의 인풋으로 사용하겠지만, 먼저 몇 가지 전처리를 설정해야 합니다:

- CSV에는 열 헤더가 없으므로, `column_names`를 수동으로 지정해야 합니다.
- `tokenizer`는 긴 헤드라인을 모델이 지원하는 최대 길이로 잘라내는 것을 포함하여 원시 텍스트를 모델이 예상하는 (숫자) 인풋으로 변환합니다.

In [ ]:
def preprocess(batch):
    """Tokenize and pre-process raw examples for training/validation"""
    result = tokenizer(batch["title"], truncation=True)
    result["label"] = batch["category"]
    return result


# Load the raw datasets:
raw_train_dataset = datasets.load_dataset(
    "csv",
    data_files=os.path.join(local_dir, "train.csv"),
    column_names=["category", "title", "content"],
    split=datasets.Split.ALL,
)
raw_test_dataset = datasets.load_dataset(
    "csv",
    data_files=os.path.join(local_dir, "test.csv"),
    column_names=["category", "title", "content"],
    split=datasets.Split.ALL,
)

# Run the tokenization/pre-processing, keeping only the output fields from preprocess()
train_dataset = raw_train_dataset.map(
    preprocess, batched=True, batch_size=1000, remove_columns=raw_train_dataset.column_names
)
test_dataset = raw_test_dataset.map(
    preprocess, batched=True, batch_size=1000, remove_columns=raw_test_dataset.column_names
)

파라미터와 사전 처리된 데이터가 로드되었으므로 모델을 훈련하고 평가할 준비가 되었습니다.

> ⏰ **Note:** 기본 `ml.t3.medium`(2 vCPU + 4 GiB RAM) Studio 인스턴스 유형에서 이 프로세스를 완료하는 데 약 20분이 소요됩니다.
>
> 기다리는 동안 [SageMaker notebook](Headline%20Classifier%20SageMaker.ipynb)으로 이동하여 이 프로세스가 SageMaker 훈련 작업으로 마이그레이션될 때 어떻게 달라지는지 살펴볼 수 있습니다.

In [ ]:
%%time

# create Trainer instance
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

# train model
trainer.train()

# evaluate model
eval_result = trainer.evaluate(eval_dataset=test_dataset)

메트릭에서 볼 수 있듯이 여기에서 학습된 모델은 정확도가 높지 않을 가능성이 높으며, 학습이 종료된 시점에서도 정확도가 빠르게 증가하고 있습니다.

## 추론에 모델 사용

모델이 학습되면 새로운 데이터에 대한 추론에 사용할 준비가 된 것입니다.

여기에서는 학습 과정에서 모델이 이미 메모리에 로드되어 있으므로 [Pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines)으로 래핑하여 쉽게 사용할 수 있습니다.

아래 셀은 사용자가 직접 뉴스 헤드라인을 입력하고 모델이 실시간으로 분류하도록 할 수 있는 대화형 위젯을 생성합니다:

In [ ]:
pipe = transformers.pipeline(
    task="text-classification",
    model=model,
    tokenizer=tokenizer,
)


def classify(text: str) -> dict:
    """Classify a headline and print the results"""
    print(pipe(text)[0])


# Either try out the interactive widget:
interaction = widgets.interact_manual(
    classify,
    text=widgets.Text(
        value="The markets were bullish after news of the merger",
        placeholder="Type a news headline...",
        description="Headline:",
        layout=widgets.Layout(width="99%"),
    ),
)
interaction.widget.children[1].description = "Classify!"

또는 코드에서 직접 파이프라인을 호출할 수도 있습니다:

In [ ]:
classify("Retailers are expanding after the recent economic growth")

## 리뷰

이 노트북에서는 일반 Jupyter 환경에서 로컬로 허깅 페이스 트랜스포머를 사용해 텍스트 분류 모델을 훈련하는 방법을 보여드렸습니다.

기본 노트북 컴퓨팅 인프라(`ml.t3.medium`)가 상당히 작았기 때문에 훈련에 시간이 오래 걸렸고 결과를 탐색해 보기 위해 조기에 중단해야 했습니다.

- 더 나은 모델을 훈련하기 위해 훈련 에포크/단계 컷오프를 확장할 수 있지만, 그러면 프로세스가 더 오래 걸립니다.
- 스튜디오 노트북을 더 높은 리소스 인스턴스(GPU 사용 가능)로 전환할 수도 있지만, 그러면 데이터 탐색이나 평가 등 실제로 모델을 학습하지 않는 시간에는 추가 리소스가 유휴 상태가 될 수 있습니다.
- 또한 훈련 과정에서 시도한 다양한 매개 변수를 추적하기 위해 실험을 수동으로 기록해야 합니다.

다음으로, [SageMaker notebook](Headline%20Classifier%20SageMaker.ipynb)으로 이동하여 필요한 만큼만 비용을 지불하면서 온디맨드 컴퓨팅을 활용하여 더 빠른 훈련과 자동 메타데이터 추적을 수행하는 데 SageMaker 훈련 작업 및 엔드포인트 배포를 활용하는 방법을 보여드리겠습니다.